In [1]:
from gurobipy import Model, GRB, quicksum
import numpy as np

In [2]:
from itertools import product

num_weeks = 9
num_consmatches_1 = 4
num_consmatches_2 = 5

def valid_pattern(pattern):
    h_count = pattern.count('H')
    a_count = pattern.count('A')
    
    # Check the total H/A counts
    if not (
        (h_count == num_consmatches_1 and a_count == num_consmatches_2) or
        (h_count == num_consmatches_2 and a_count == num_consmatches_1)
    ):
        return False
    
    # Check for "breaks" only between odd/even round pairs
    for i in range(0, len(pattern), 2):
        if i + 1 < len(pattern):
            if pattern[i] == pattern[i + 1]:
                return False
    
    return True

all_patterns = product('HA', repeat=num_weeks)
valid_patterns = [
    ''.join(p) for p in all_patterns
    if valid_pattern(p)
]

num_patterns = len(valid_patterns)
print(f"Total valid patterns: {len(valid_patterns)}")
print(valid_patterns)


Total valid patterns: 32
['HAHAHAHAH', 'HAHAHAHAA', 'HAHAHAAHH', 'HAHAHAAHA', 'HAHAAHHAH', 'HAHAAHHAA', 'HAHAAHAHH', 'HAHAAHAHA', 'HAAHHAHAH', 'HAAHHAHAA', 'HAAHHAAHH', 'HAAHHAAHA', 'HAAHAHHAH', 'HAAHAHHAA', 'HAAHAHAHH', 'HAAHAHAHA', 'AHHAHAHAH', 'AHHAHAHAA', 'AHHAHAAHH', 'AHHAHAAHA', 'AHHAAHHAH', 'AHHAAHHAA', 'AHHAAHAHH', 'AHHAAHAHA', 'AHAHHAHAH', 'AHAHHAHAA', 'AHAHHAAHH', 'AHAHHAAHA', 'AHAHAHHAH', 'AHAHAHHAA', 'AHAHAHAHH', 'AHAHAHAHA']


In [3]:
solution_found = {'v': {(2, 4, 4): 1.0,
  (2, 6, 5): 1.0,
  (2, 13, 6): 1.0,
  (2, 16, 7): 1.0,
  (2, 25, 2): 1.0,
  (2, 27, 3): 1.0,
  (2, 29, 0): 1.0,
  (2, 31, 1): 1.0,
  (3, 4, 6): 1.0,
  (3, 6, 4): 1.0,
  (3, 13, 7): 1.0,
  (3, 16, 0): 1.0,
  (3, 25, 3): 1.0,
  (3, 27, 1): 1.0,
  (3, 29, 5): 1.0,
  (3, 31, 2): 1.0,
  (4, 6, 7): 1.0,
  (4, 13, 3): 1.0,
  (4, 16, 1): 1.0,
  (4, 25, 5): 1.0,
  (4, 27, 0): 1.0,
  (4, 29, 2): 1.0,
  (6, 13, 2): 1.0,
  (6, 16, 6): 1.0,
  (6, 25, 0): 1.0,
  (6, 29, 1): 1.0,
  (6, 31, 3): 1.0,
  (13, 16, 4): 1.0,
  (13, 25, 1): 1.0,
  (13, 27, 5): 1.0,
  (13, 31, 0): 1.0,
  (16, 27, 2): 1.0,
  (16, 29, 3): 1.0,
  (16, 31, 5): 1.0,
  (25, 27, 7): 1.0,
  (25, 29, 4): 1.0,
  (25, 31, 6): 1.0,
  (27, 29, 6): 1.0,
  (27, 31, 4): 1.0,
  (29, 31, 7): 1.0},
 'u': {2: 1.0,
  3: 1.0,
  4: 1.0,
  6: 1.0,
  13: 1.0,
  16: 1.0,
  25: 1.0,
  27: 1.0,
  29: 1.0,
  31: 1.0}}

In [4]:
selected_patterns = solution_found['u']
schedule = solution_found['v']

In [5]:
import pandas as pd

# Extract weeks 
weeks = sorted({t for _, _, t in schedule.keys()})
calendar_data = {f"Week {t+1}": [] for t in weeks}

# Populate calendar data for each week
for t in weeks:
    week_pairs = []
    for (i, j, w) in schedule:
        if w == t:
            week_pairs.append(f"Pattern {i} vs Pattern {j}")
    calendar_data[f"Week {t+1}"] = week_pairs


calendar_df = pd.DataFrame(calendar_data)
calendar_df

,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8
0,Pattern 2 vs Pattern 29,Pattern 2 vs Pattern 31,Pattern 2 vs Pattern 25,Pattern 2 vs Pattern 27,Pattern 2 vs Pattern 4,Pattern 2 vs Pattern 6,Pattern 2 vs Pattern 13,Pattern 2 vs Pattern 16
1,Pattern 3 vs Pattern 16,Pattern 3 vs Pattern 27,Pattern 3 vs Pattern 31,Pattern 3 vs Pattern 25,Pattern 3 vs Pattern 6,Pattern 3 vs Pattern 29,Pattern 3 vs Pattern 4,Pattern 3 vs Pattern 13
2,Pattern 4 vs Pattern 27,Pattern 4 vs Pattern 16,Pattern 4 vs Pattern 29,Pattern 4 vs Pattern 13,Pattern 13 vs Pattern 16,Pattern 4 vs Pattern 25,Pattern 6 vs Pattern 16,Pattern 4 vs Pattern 6
3,Pattern 6 vs Pattern 25,Pattern 6 vs Pattern 29,Pattern 6 vs Pattern 13,Pattern 6 vs Pattern 31,Pattern 25 vs Pattern 29,Pattern 13 vs Pattern 27,Pattern 25 vs Pattern 31,Pattern 25 vs Pattern 27
4,Pattern 13 vs Pattern 31,Pattern 13 vs Pattern 25,Pattern 16 vs Pattern 27,Pattern 16 vs Pattern 29,Pattern 27 vs Pattern 31,Pattern 16 vs Pattern 31,Pattern 27 vs Pattern 29,Pattern 29 vs Pattern 31


In [6]:
# Create the list of teams and ranking
teams = [
    "Brazil", "Argentina", "Uruguay", 
    "Ecuador", "Peru", "Colombia", 
    "Chile", "Paraguay", "Bolivia", "Venezuela"
]

ranking = {
    1: teams[:2],  
    2: teams[2:6],  
    3: teams[6:]    
}

# Assign a unique number to each team
team_numbers = {i : team for i, team in enumerate(teams)}

# Create a reverse mapping to look up the number of a team
team_to_number = {team: number for number, team in team_numbers.items()}


stronger_teams = {}

# Iterate through each team and determine stronger teams
for position, teams_at_position in ranking.items():
    for team in teams_at_position:
        team_number = team_to_number[team]  
        stronger_teams[team_number] = [
            team_to_number[stronger_team]
            for higher_position in range(1, position)  
            for stronger_team in ranking[higher_position]
        ]


print("Team Numbers:", team_numbers)
print("Stronger Teams:", stronger_teams)

Team Numbers: {0: 'Brazil', 1: 'Argentina', 2: 'Uruguay', 3: 'Ecuador', 4: 'Peru', 5: 'Colombia', 6: 'Chile', 7: 'Paraguay', 8: 'Bolivia', 9: 'Venezuela'}
Stronger Teams: {0: [], 1: [], 2: [0, 1], 3: [0, 1], 4: [0, 1], 5: [0, 1], 6: [0, 1, 2, 3, 4, 5], 7: [0, 1, 2, 3, 4, 5], 8: [0, 1, 2, 3, 4, 5], 9: [0, 1, 2, 3, 4, 5]}


In [7]:
from gurobipy import Model, GRB, quicksum

# Initialize the model
m = Model("Team_to_Pattern_Allocation")

## Parameters and Sets
# U = set of teams
U = range(10)   

# S = set of selected patterns (already chosen in a previous step)
S = selected_patterns 

# T = set of rounds/weeks
T = range(9)


# Given calendar
calendar = schedule

## Decision variables
# w_{i,s} = 1 if team i is assigned to pattern s
w = m.addVars(U, S, vtype=GRB.BINARY, name="w")

# x_{i,j,t} = 1 if team i faces team j in round t
x = m.addVars(U, U, T, vtype=GRB.BINARY, name="x")

# q_{i,k} = 1 if there's a carry-over effect for team i between weeks k and k+1
q = m.addVars(U, range(len(T)-1), vtype=GRB.BINARY, name="q")


# Objective: Minimize carry-over effect
m.setObjective(quicksum(q[i, k] for i in U for k in range(len(T)-1)), GRB.MINIMIZE)



Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-04


In [8]:
# (2b) Each team i in U must be assigned exactly one pattern
for i in U:
    m.addConstr(
        quicksum(w[i, s] for s in S) == 1,
        name=f"(2b)_team_{i}_pattern_assign"
    )

# (2c) Each pattern s in S must be assigned to exactly one team
for s in S:
    m.addConstr(
        quicksum(w[i, s] for i in U) == 1,
        name=f"(2c)_pattern_{s}_unique_team"
    )


# (2d) Link pattern assignments to actual matches x_{i,j,t}
# For every (s_i, s_j, t) in the 'calendar', ensure x_{i,j,t} = 1
# only if i has pattern s_i and j has pattern s_j, etc.
#
# Below is one way: x_{i,j,t} >= w_{i,s_i} + w_{j,s_j} - 1
# so if both w_{i,s_i} = 1 and w_{j,s_j} = 1, x_{i,j,t} can be forced to 1.
# The exact shape of your linking constraints may differ, but here's the rename:
for (s_i, s_j, t) in calendar.keys():
    for i in U:
        for j in U:
            if i < j:
                m.addConstr(
                    x[i, j, t] >= w[i, s_i] + w[j, s_j] - 1,
                    name=f"(2d)_link_{i}_{j}_{t}"
                )

# (2e) Each pair (i, j) meets exactly once across all T
for i in U:
    for j in U:
        if i < j:
            m.addConstr(
                quicksum(x[i, j, t] for t in T) == 1,
                name=f"(2e)_pair_once_{i}_{j}"
            )

# (2f) Each team plays exactly once per round t
for t in T:
    for i in U:
        m.addConstr(
            quicksum(x[i, j, t] for j in U if j > i) +
            quicksum(x[j, i, t] for j in U if j < i) == 1,
            name=f"(2f)_one_match_{i}_{t}"
        )

# Prevent i vs i, or i>j duplication:
for i in U:
    for j in U:
        if i >= j:
            for t in T:
                x[i, j, t].ub = 0

# (2g), (2h), (2i): Carry-over effect constraints
# We want to set q_{i,k} = 1 if team i faces "stronger" opponents in both week k and k+1.
# The code below says:
#   x[i,j,k] + x[i,j,k+1] + x[j,i,k] + x[j,i,k+1] <= 1 + q[i,k]
#   and also
#   x[i,j,k] + x[j,i,k] >= q[i,k],
#   x[i,j,k+1] + x[j,i,k+1] >= q[i,k].
#
# This effectively forces q[i,k] to 1 if i faces a "stronger" team in both k and k+1.
for i in U:
    for k in range(len(T)-1):
        # (2g) Upper bound
        m.addConstr(
            quicksum(x[i, j, k]   for j in stronger_teams[i] if j > i) +
            quicksum(x[j, i, k]   for j in stronger_teams[i] if j < i) +
            quicksum(x[i, j, k+1] for j in stronger_teams[i] if j > i) +
            quicksum(x[j, i, k+1] for j in stronger_teams[i] if j < i)
            <= 1 + q[i, k],
            name=f"(2g)_carry_over_upper_{i}_{k}"
        )
        # (2h) Lower bound #1
        m.addConstr(
            quicksum(x[i, j, k] for j in stronger_teams[i] if j > i) +
            quicksum(x[j, i, k] for j in stronger_teams[i] if j < i)
            >= q[i, k],
            name=f"(2h)_carry_over_lower1_{i}_{k}"
        )
        # (2i) Lower bound #2
        m.addConstr(
            quicksum(x[i, j, k+1] for j in stronger_teams[i] if j > i) +
            quicksum(x[j, i, k+1] for j in stronger_teams[i] if j < i)
            >= q[i, k],
            name=f"(2i)_carry_over_lower2_{i}_{k}"
        )


m.optimize()


if m.status == GRB.OPTIMAL:
    print("Optimal solution found.")

    # w_{i,s} solution
    w_sol = {
        (i, s): w[i, s].X
        for i in U
        for s in S
        if w[i, s].X > 0.5
    }

    # x_{i,j,t} solution
    x_sol = {
        (i, j, t): x[i, j, t].X
        for i in U
        for j in U
        for t in T
        if x[i, j, t].X > 0.5
    }

    # q_{i,k} solution
    q_sol = {
        (i, k): q[i, k].X
        for i in U
        for k in range(len(T)-1)
        if q[i, k].X > 0.5
    }

    print("Team-to-pattern allocation (w_{i,s}=1):", w_sol)
    print("Carry-over effect (q_{i,k}=1):", q_sol)
    print("Matches between teams (x_{i,j,t}=1):", x_sol)

else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2195 rows, 1080 columns and 8079 nonzeros
Model fingerprint: 0xd9cd43ac
Variable types: 0 continuous, 1080 integer (1080 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]


  RHS range        [1e+00, 1e+00]
Presolve removed 48 rows and 511 columns
Presolve time: 0.02s
Presolved: 2147 rows, 569 columns, 8031 nonzeros
Variable types: 0 continuous, 569 integer (569 binary)
Found heuristic solution: objective 14.0000000

Root relaxation: objective 8.000000e+00, 506 iterations, 0.04 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.0000000    8.00000  0.00%     -    0s

Explored 1 nodes (1143 simplex iterations) in 0.19 seconds (0.08 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: 8 14 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+00, best bound 8.000000000000e+00, gap 0.0000%
Optimal solution found.
Team-to-pattern allocation (w_{i,s}=1): {(0, 31): 1.0, (1, 29): 1.0, (2, 25): 1.0, (3, 27): 1.0, (4, 16): 1.0, (5, 13): 1.0, (6, 6): 1.0, 

In [10]:
pattern_to_team = {team_numbers[team] : p for (team, p), value in w_sol.items() if value > 0.5}
weeks = {}

for (team1, team2, week), value in x_sol.items():
    if value == 1.0:  
        if week not in weeks:
            weeks[week] = []
        key = pattern_to_team[team_numbers[team1]]  
        if valid_patterns[key][week] == 'H':
            match = f"{team_numbers[team1]} vs {team_numbers[team2]}"
        else:
            match = f"{team_numbers[team2]} vs {team_numbers[team1]}"
        if match not in weeks[week]:
            weeks[week].append(match)


sorted_weeks_example = sorted(weeks.keys())

max_matches_example = max(len(matches) for matches in weeks.values())

formatted_data_example = {
    f"Week {week}": weeks[week] + [None] * (max_matches_example - len(weeks[week]))
    for week in sorted_weeks_example
}

formatted_df_example = pd.DataFrame(formatted_data_example)
formatted_df_example

,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8
0,Paraguay vs Brazil,Brazil vs Ecuador,Chile vs Brazil,Brazil vs Argentina,Peru vs Brazil,Brazil vs Bolivia,Colombia vs Brazil,Brazil vs Venezuela,Uruguay vs Brazil
1,Uruguay vs Argentina,Argentina vs Colombia,Paraguay vs Argentina,Uruguay vs Venezuela,Ecuador vs Argentina,Argentina vs Venezuela,Argentina vs Peru,Chile vs Argentina,Bolivia vs Argentina
2,Chile vs Ecuador,Uruguay vs Peru,Colombia vs Uruguay,Ecuador vs Colombia,Uruguay vs Bolivia,Chile vs Uruguay,Uruguay vs Paraguay,Ecuador vs Uruguay,Paraguay vs Ecuador
3,Venezuela vs Peru,Paraguay vs Chile,Venezuela vs Ecuador,Paraguay vs Peru,Chile vs Colombia,Peru vs Ecuador,Bolivia vs Ecuador,Colombia vs Peru,Peru vs Chile
4,Colombia vs Bolivia,Venezuela vs Bolivia,Peru vs Bolivia,Bolivia vs Chile,Venezuela vs Paraguay,Colombia vs Paraguay,Venezuela vs Chile,Bolivia vs Paraguay,Venezuela vs Colombia
